In [1]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn

from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay, f1_score
from sklearn.model_selection import StratifiedKFold, TimeSeriesSplit
from tqdm import tqdm, trange

seed=42
FTWs = [720, 540, 360, 180, 60, 30, 15, 5, 3, 2, 1, 0, 0]
ftw_window = 10

/home/simon/miniconda3/envs/casas/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [3]:
features = np.load('./ftw_data/features.npy')
activities = np.load('./ftw_data/activities.npy')

In [4]:
tsv = TimeSeriesSplit(n_splits=3)
kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=seed)
k = 0

# Problem about the time series split since it would have overlap with the previous
for train, test in list(tsv.split(features, activities))[-1:]:
    print('X_train shape:', features[train].shape)
    print('y_train shape:', activities[train].shape)

X_train shape: (3347, 10, 56)
y_train shape: (3347, 17)


In [5]:
class LSTM(nn.Module):
    
    def __init__(self,input_dim,hidden_dim,output_dim,layer_num):
        super(LSTM,self).__init__()
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.lstm = nn.LSTM(input_dim,hidden_dim,layer_num,batch_first=False)
        self.fc = nn.Linear(hidden_dim,output_dim)
        self.sigmoid = nn.Sigmoid()
        # self.bn = nn.BatchNorm1d(32)
        
    def forward(self,inputs):
        # x = self.bn(inputs)
        lstm_out,(hn,cn) = self.lstm(inputs)
        out = self.fc(lstm_out)
        # print(out.shape)
        return self.sigmoid(out)

In [6]:
ensemble_activities = activities.T

### Mean

In [7]:
def preprocess_features(features, method):
    if method not in ['mean', 'mean_std', 'mean_wth_weight', 'mean_std_max_min']:
        raise ValueError('Please double check the method parameter')

    preprocessed_features = np.array([])
    if method == 'mean':
        return features.mean(axis=1)
    elif method == 'mean_std':
        preprocess_features = np.array(features.mean(axis=1))
        print(preprocess_features.shape)
        preprocess_features = np.concatenate([preprocess_features, features.std(axis=1)], axis=1)
        print(preprocess_features.shape)
        return preprocess_features
    elif method == 'mean_wth_weight':
        out = []
        for instance in features:
            out.append(np.sum([vector*((i+1)/10) for i, vector in enumerate(instance)], axis=0))
        return np.array(out)      

In [8]:
FTWs
sum = 0
for i in range(ftw_window):
    l4, l3, l2, l1 = FTWs[i:i+4]
    sum += (l3 - l2)
sum

540

In [9]:
mean_features = preprocess_features(features, 'mean_wth_weight')

input_size = mean_features.shape[1]

print(input_size)

n_hidden = 512
n_categories = 1
n_layer = 3
rnn = LSTM(input_size,n_hidden,n_categories,n_layer)

rnn.to(device)

56


LSTM(
  (lstm): LSTM(56, 512, num_layers=3)
  (fc): Linear(in_features=512, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [10]:
print(input.shape)
ensemble_activities.shape

AttributeError: 'function' object has no attribute 'shape'

In [12]:
import torch.optim as optim
import time
import math

criterion = nn.BCELoss()
learning_rate = 0.0005
optimizer = optim.Adam(rnn.parameters(),lr=learning_rate)
#scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10000, gamma=0.1)

n_iters = 1000
print_every = 10
plot_every = 10
batch_size = 256

# Keep track of losses for plotting
current_loss = 0
all_losses = []
accur = []

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

sleep_activities = ensemble_activities[0]
train, test = list(tsv.split(mean_features, sleep_activities))[-1]

print('X_train shape:', mean_features[train].shape)
print('y_train shape:', sleep_activities[train].shape)
train_features, train_activities = mean_features[train], sleep_activities[train]
dev_features, dev_activities = mean_features[test], sleep_activities[test]

start = time.time()

for iter in trange(1, n_iters + 1):
    
    train_labels, train_tensor = torch.tensor(train_activities, dtype=torch.float32), torch.tensor(train_features, dtype=torch.float32)
    dev_labels, dev_tensor = torch.tensor(dev_activities, dtype=torch.float32), torch.tensor(dev_features, dtype=torch.float32)

    train_tensor = train_tensor.to(device)
    train_labels = train_labels.to(device).unsqueeze(1)

    dev_tensor = dev_tensor.to(device)
    dev_labels = dev_labels.to(device).unsqueeze(1)
    
    optimizer.zero_grad()
    
    output = rnn(train_tensor)
    loss = criterion(output, train_labels)
    loss.backward()
    optimizer.step()
    #scheduler.step()
    
    prediction = rnn(dev_tensor)
    pred_cpu =  prediction.reshape(-1).cpu().detach().numpy().round()
    acc = f1_score(dev_activities, pred_cpu)
    #backprop

    if iter%print_every == 0:
        accur.append(acc)
        print("epoch {}\tloss : {}\t accuracy : {}".format(iter,loss,acc))
        print(classification_report(dev_activities, pred_cpu))
    
    current_loss += loss.item()
    
    # category = LABELS[int(category_tensor[0])]

    # # Print iter number, loss, name and guess
    # if iter % print_every == 0:
    #     guess, guess_i = categoryFromOutput(output)
    #     correct = '✓' if guess == category else '✗ (%s)' % category
    #     print('%d %d%% (%s) %.4f  / %s %s' % (iter, iter / n_iters * 100, timeSince(start), loss, guess, correct))
        
    # Add current loss avg to list of losses
    if iter % plot_every == 0:
        all_losses.append(current_loss / plot_every)
        current_loss = 0

X_train shape: (3347, 56)
y_train shape: (3347,)


  1%|          | 10/1000 [00:02<04:34,  3.60it/s]

epoch 10	loss : 0.32651543617248535	 accuracy : 0.6504065040650405
              precision    recall  f1-score   support

         0.0       0.94      0.92      0.93       937
         1.0       0.63      0.67      0.65       178

    accuracy                           0.88      1115
   macro avg       0.78      0.80      0.79      1115
weighted avg       0.89      0.88      0.89      1115



  2%|▏         | 20/1000 [00:05<04:32,  3.60it/s]

epoch 20	loss : 0.28317540884017944	 accuracy : 0.694300518134715
              precision    recall  f1-score   support

         0.0       0.95      0.92      0.94       937
         1.0       0.64      0.75      0.69       178

    accuracy                           0.89      1115
   macro avg       0.80      0.84      0.82      1115
weighted avg       0.90      0.89      0.90      1115



...

epoch 980	loss : 0.0004800430906470865	 accuracy : 0.7135416666666665
              precision    recall  f1-score   support

         0.0       0.95      0.93      0.94       937
         1.0       0.67      0.77      0.71       178

    accuracy                           0.90      1115
   macro avg       0.81      0.85      0.83      1115
weighted avg       0.91      0.90      0.90      1115



 99%|█████████▉| 990/1000 [04:36<00:02,  3.53it/s]

epoch 990	loss : 0.00047575627104379237	 accuracy : 0.7135416666666665
              precision    recall  f1-score   support

         0.0       0.95      0.93      0.94       937
         1.0       0.67      0.77      0.71       178

    accuracy                           0.90      1115
   macro avg       0.81      0.85      0.83      1115
weighted avg       0.91      0.90      0.90      1115



100%|██████████| 1000/1000 [04:39<00:00,  3.58it/s]

epoch 1000	loss : 0.00047154241474345326	 accuracy : 0.7135416666666665
              precision    recall  f1-score   support

         0.0       0.95      0.93      0.94       937
         1.0       0.67      0.77      0.71       178

    accuracy                           0.90      1115
   macro avg       0.81      0.85      0.83      1115
weighted avg       0.91      0.90      0.90      1115



In [ ]:
all_losses

In [ ]:
# Score accuracy if at least one is true

category_tensor[0].dtype

### TODO
- Apply the model to other activities and plot heatmap
- Check any paper talked about approach for weight for window size
- Multi-label, consider the correlation between labels
- Consider more data from hh1xx dataset

### Mean and std

### Mean, std, max, min and crossing rate

### Mean with weight for each time window